In [1]:
!pip install --upgrade xgboost optuna scikit-learn pandas

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 560.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 548.2 kB/s eta 0:00:00a 0:00:01


In [2]:
import pandas as pd
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from math import sqrt

from sklearn.metrics import mean_squared_error

/Users/arthparashar/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv("/Users/arthparashar/Documents/ML_podcast_time_project/final_train.csv")

In [5]:
df

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Episode_Title_num,Ad_Density,Popularity_Diff,Popularity_Interaction,Host_Popularity_squared,Popularity_Average
0,0,34,98,63.84,9,74.81,4,3,3.0,0.0,1,31.41998,98,0.000000,71.81,224.43,5596.5361,38.905
1,1,24,19,119.80,1,66.95,2,0,3.0,2.0,-1,88.01241,26,0.016694,63.95,200.85,4482.3025,34.975
2,2,40,8,73.90,2,69.97,5,1,3.0,0.0,-1,44.92531,16,0.000000,66.97,209.91,4895.8009,36.485
3,3,10,40,67.17,8,57.22,1,2,3.0,2.0,1,46.27824,45,0.029775,54.22,171.66,3274.1284,30.110
4,4,31,85,110.51,3,80.07,1,0,3.0,3.0,0,75.61031,86,0.027147,77.07,240.21,6411.2049,41.535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,26,18,75.66,2,69.36,2,2,3.0,0.0,-1,56.87058,25,0.000000,66.36,208.08,4810.8096,36.180
749996,749996,2,14,75.75,0,35.21,2,3,3.0,2.0,0,45.46242,21,0.026402,32.21,105.63,1239.7441,19.105
749997,749997,28,47,30.98,4,78.58,4,2,3.0,0.0,-1,15.26000,51,0.000000,75.58,235.74,6174.8164,40.790
749998,749998,41,42,108.98,4,45.39,4,2,3.0,0.0,-1,100.72939,47,0.000000,42.39,136.17,2060.2521,24.195


In [6]:

df.drop(columns=["id"],inplace=True)

In [30]:
df = df.drop(columns=["Podcast_Name", "Episode_Title"])

In [31]:

X = df.drop(columns=["Listening_Time_minutes"])
y = df["Listening_Time_minutes"]

In [32]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
print("X_train shape:", X_train.shape)
print("X_valid shape:", X_valid.shape)
print("y_train shape:", y_train.shape)
print("y_valid shape:", y_valid.shape)

X_train shape: (600000, 14)
X_valid shape: (150000, 14)
y_train shape: (600000,)
y_valid shape: (150000,)


In [ ]:

def objective(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_lambda": trial.suggest_float("lambda", 1e-5, 1.0),
        "reg_alpha": trial.suggest_float("alpha", 1e-5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "objective": "reg:squarederror",
        "random_state": 42,
    }

    model = xgb.XGBRegressor(**params)

    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)

    preds = model.predict(X_valid)
    rmse = sqrt(mean_squared_error(y_valid, preds))  
    return rmse



In [ ]:

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)  

[I 2025-04-12 23:03:33,135] A new study created in memory with name: no-name-1ab6552f-abeb-47ea-b280-dd59c6f2437f
[I 2025-04-12 23:03:39,092] Trial 0 finished with value: 13.059028851061239 and parameters: {'max_depth': 4, 'learning_rate': 0.21888988584305052, 'subsample': 0.6386342984164592, 'colsample_bytree': 0.5898493958762712, 'lambda': 0.23405611791682154, 'alpha': 0.8780635717957419, 'min_child_weight': 1, 'gamma': 3.909498455731277, 'n_estimators': 510}. Best is trial 0 with value: 13.059028851061239.
[I 2025-04-12 23:03:58,493] Trial 1 finished with value: 13.47441870516541 and parameters: {'max_depth': 11, 'learning_rate': 0.26545025644426146, 'subsample': 0.9969508333927368, 'colsample_bytree': 0.9302888968127482, 'lambda': 0.12321169599483901, 'alpha': 0.5989177059714669, 'min_child_weight': 5, 'gamma': 2.7652616704462076, 'n_estimators': 989}. Best is trial 0 with value: 13.059028851061239.
[I 2025-04-12 23:04:08,708] Trial 2 finished with value: 13.127242193936576 and par

In [37]:
print("Best Parameters:")
print(study.best_params)

print("\nBest Score (RMSE):")
print(study.best_value)

print("\nBest Trial:")
print(study.best_trial)

Best Parameters:
{'max_depth': 15, 'learning_rate': 0.02284007073883936, 'subsample': 0.6025836878781773, 'colsample_bytree': 0.7557064492201065, 'lambda': 0.2168416825417549, 'alpha': 0.43124918040036164, 'min_child_weight': 1, 'gamma': 4.098130390872883, 'n_estimators': 376}

Best Score (RMSE):
12.77128484012427

Best Trial:
FrozenTrial(number=31, state=TrialState.COMPLETE, values=[12.77128484012427], datetime_start=datetime.datetime(2025, 4, 12, 23, 35, 53, 509368), datetime_complete=datetime.datetime(2025, 4, 12, 23, 36, 26, 675782), params={'max_depth': 15, 'learning_rate': 0.02284007073883936, 'subsample': 0.6025836878781773, 'colsample_bytree': 0.7557064492201065, 'lambda': 0.2168416825417549, 'alpha': 0.43124918040036164, 'min_child_weight': 1, 'gamma': 4.098130390872883, 'n_estimators': 376}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'max_depth': IntDistribution(high=15, log=False, low=3, step=1), 'learning_rate': FloatDistribution(high=0.3, log=Fa

In [38]:
best_params={

    'max_depth': 14,
    'learning_rate': 0.023729272005270058,
    'subsample': 0.8968090616429454,
    'colsample_bytree': 0.8492576547683933,
    'lambda': 0.9990102695713851,
    'alpha': 0.38709502558228537,
    'min_child_weight': 6,
    'gamma': 3.823134356331538,
    'n_estimators': 819
}

In [39]:
code = xgb.XGBRegressor(**best_params)

In [40]:
code.fit(X_train,y_train)

XGBRegressor(alpha=0.38709502558228537, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8492576547683933, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=3.823134356331538,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.9990102695713851,
             learning_rate=0.023729272005270058, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=14, max_leaves=None,
             min_child_weight=6, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=819, n_jobs=None, ...)

In [41]:
code

XGBRegressor(alpha=0.38709502558228537, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8492576547683933, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=3.823134356331538,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.9990102695713851,
             learning_rate=0.023729272005270058, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=14, max_leaves=None,
             min_child_weight=6, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=819, n_jobs=None, ...)

In [42]:
y_out = code.predict(X_valid)

In [43]:
test = pd.read_csv('/Users/arthparashar/Documents/ML_podcast_time_project/final_test.csv')

In [53]:
test.head()


,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Episode_Title_num,Ad_Density,Popularity_Diff,Popularity_Interaction,Host_Popularity_squared,Popularity_Average
0,11,71,78.96,2,38.11,2,1,3.0,1.0,0,73,0.012664,35.11,114.33,1452.3721,20.555
1,36,16,27.87,5,71.29,3,2,3.0,0.0,0,23,0.000000,68.29,213.87,5082.2641,37.145
2,24,3,69.10,1,67.89,0,1,3.0,0.0,1,11,0.000000,64.89,203.67,4609.0521,35.445
3,4,71,115.39,1,23.40,3,2,3.0,2.0,1,73,0.017332,20.40,70.20,547.5600,13.200
4,27,46,72.32,4,58.10,6,2,3.0,2.0,0,50,0.027654,55.10,174.30,3375.6100,30.550


In [55]:
if 'id' in test.columns:
    test.drop(columns=['id'], inplace=True)


In [56]:
test_cleaned = test.drop(columns=['Podcast_Name', 'Episode_Title'])
y_xg = code.predict(test_cleaned)


In [57]:
y_xg

array([53.626034, 19.96012 , 46.8305  , ...,  7.523348, 70.61406 ,
       60.054253], dtype=float32)

In [58]:
sample = pd.read_csv('/Users/arthparashar/Documents/ML_podcast_time_project/dirichlet_blended_4_combined.csv')

In [59]:
sample['Listening_Time_minutes'] = y_xg

In [60]:
sample.to_csv('final_model_trained_using_xgboost.csv',index =False)

In [ ]:

import joblib
joblib.dump(code, "model.pkl")

['model.pkl']